In [1]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
import csv
import xlrd
import os
import os.path
from pathlib2 import Path
from decimal import *
%matplotlib inline

## import namd & gmx data

In [7]:
gmx_surface_areas = ['SA19-1_20ns', 'SA19-2_100ns', 'SA19-3_20ns',
                 'SA19.5_100ns', 'SA19.5-2_20ns', 'SA19.5-3_20ns',
                 'SA20_20ns', 'SA20-2_20ns', 'SA20-3_100ns',
                 'SA21_100ns', 'SA21-2_20ns', 'SA21-3_20ns', 
                 'SA22_20ns', 'SA22-2_20ns', 'SA22-3_100ns', 
                 'SA24_20ns', 'SA24-2_20ns', 'SA24-3_100ns']
namd_surface_areas = ['SA19','SA19-2', 'SA19-3', 
                   'SA19.5','SA19.5-2', 'SA19.5-3',
                   'SA20','SA20-2', 'SA20-3',
                   'SA20.5', 'SA20.5-2', 'SA20.5-3',
                   'SA21','SA21-2', 'SA21-3',
                   'SA22','SA22-2', 'SA22-3',
                   'SA23','SA23-2', 'SA23-3',
                   'SA24', 'SA24-2', 'SA24-3']
namd_filenames = ['step7.1', 'step7.2', 'step7.21', 'step7.3', 'step7.31', 'step7.32', 'step7.33', 'step7.34',
               'step7.35','step7.4', 'step7.41', 'step7.42', 'step7.43', 'step7.44', 'step7.45']
gmx_filenames = []
for g in range(1,21):
    gmx_filenames.append('step9_'+str(g))
filedir_g = '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/'
filedir_n = '/gpfs/amarolab/abbyflabby/MIX/MIX1234_NaCl0.4_NAMD/'
logfiles_n = []
logfiles_g = []
trr_files = []
dcd_files = []

for area in namd_surface_areas:
    logchunk = []
    dcdchunk = []
    for i in namd_filenames:
        log = Path(filedir_n+area+'/charmm-gui/namd/'+i+'_production.log')
        dcd = Path(filedir_n+area+'/charmm-gui/namd/'+i+'_production.dcd')
        if log.exists():
            logchunk.append(filedir_n+area+'/charmm-gui/namd/'+i+'_production.log')
        if dcd.exists():
            dcdchunk.append(filedir_n+area+'/charmm-gui/namd/'+i+'_production.dcd')
    if len(logchunk)>0:
        logfiles_n.append(logchunk)
        dcd_files.append(dcdchunk)
#xvgfiles = [] #list of xvg filenames
pres = []
times = []
areas = []
trr_files = []
#for narea in n_surface_areas:
#    logchunk = []
    
for area in gmx_surface_areas:
    xvgchunk = []
    trajchunk = []
    for i in range(1, 21):
        xvg = Path(filedir_g+area+'/step9_'+str(i)+'.xvg')
        traj = Path(filedir_g+area+'/step9_'+str(i)+'.trr')
        if xvg.exists():
            xvgchunk.append(filedir_g+area+'/step9_'+str(i)+'.xvg')
        if traj.exists():
            trajchunk.append(filedir_g+area+'/step9_'+str(i))
    #print(xvgchunk)
    for p, x in enumerate(xvgchunk):
        if p == 0:
            #arr = np.array(timesteps)
            #timesteps in nanoseconds
            timesteps = np.genfromtxt(x, dtype='float', skip_header=24)[:,0]/1000
            #np.append(timesteps, new_ts)
            pressures = np.genfromtxt(x, dtype='float', skip_header=24)[:,1]/20
            #np.append(pressures, new_pr)
            #print(pressures)
        else:
            time = np.genfromtxt(x,dtype='float', skip_header=24)[:,0]/1000
            #print(time)
            timesteps = np.append(timesteps, time)
      #      #print(time)
      #      #t = time.tolist()
      #      timesteps.append(time)
      #      print(timesteps)
            pr = np.genfromtxt(x, dtype='float', skip_header=24)[:,1]/20
            pressures = np.append(pressures, pr)
    if len(xvgchunk)>0:
        pres.append(70.5-np.array(pressures))
        times.append(timesteps)
        areas.append(area)
        logfiles_g.append(xvgchunk)
        trr_files.append(trajchunk)
#print(len(xvgfiles))
#print(xvgfiles)
#print(len(times[1]))
#print(times[1][-1])
#print(len(pres[1]))
#print(len(times))
#print(areas)
print(trr_files)

[['/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-1_20ns/step9_1'], ['/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_1', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_2', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_3', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_4', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_5', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_6', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_7', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_8', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_9', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_10', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_11', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_12', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_13', '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100n

## strip trajectories of water

In [ ]:
def genStripScript():
    with open(filedir_g+'/strip_script.sh', 'w') as file:
        for count, area in enumerate(areas):
            for step, t in enumerate(trr_files[count]):
                file.write('gmx trjconv -f '+filedir_g+area+'/step9_'+str(step+1)+'.trr -o '+filedir_g+area+'/step9_'+str(step+1)+'_stripped.xtc -s '+filedir_g+area+'/step9_'+str(step+1)+'.tpr\n')
#genStripScript()

In [ ]:
genStripScript()

## load gmx trajectories

In [12]:
gmx_traj = []
namd_traj = []
xtcs = []
for trr in trr_files:
    #print(trr_files)
    chunk = []
    for t in trr:
        chunk.append(t+'_stripped.xtc')
    xtcs.append(chunk)
#print(xtcs)
print(len(xtcs))
#print(xtcs)
#traj1 = md.load(trr_files[0], top=filedir_g+areas[0]+'/step9_1.gro')
#for i,g in enumerate(xtcs):
    #traj = md.load(g, top=filedir_g+areas[i]+'/step9_1.gro')
    #gmx_traj.append(traj)#
#print(gmx_traj)

16


In [14]:
ct = 0 
for x in xtcs:
    chunk = []
    #print(x)
    print(filedir_g+areas[ct]+'/step9_1_stripped.pdb')
    traj = md.load(x, top=filedir_g+areas[ct]+'/step9_1_stripped.pdb')
    #chunk.append(traj)
    gmx_traj.append(traj)
    ct+=1
print(gmx_traj)

/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-1_20ns/step9_1_stripped.pdb
/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_1_stripped.pdb
/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19-3_20ns/step9_1_stripped.pdb
/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19.5_100ns/step9_1_stripped.pdb
/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19.5-2_20ns/step9_1_stripped.pdb
/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA19.5-3_20ns/step9_1_stripped.pdb
/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA20_20ns/step9_1_stripped.pdb
/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA20-2_20ns/step9_1_stripped.pdb
/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA20-3_100ns/step9_1_stripped.pdb
/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA21_100ns/step9_1_stripped.pdb
/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA21-2_20ns/step9_1_stripped.pdb
/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA21-3_20ns/step9_1_stripped.pdb
/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/SA22_20ns/step9_1_stripped.pdb
/gpfs/amar

## obtain trajectories for upper and lower leaflets

In [ ]:
count = 0
lipidTrajArray = []
lipids = []
ptTrajArray = []
for traj in gmx_traj:
    #traj = md.load(dcds[count], top=filedir1+"SA"+i+"/charmm-gui/step5_assembly.xplor_ext.psf")
    #traj = md.load(dcds[count], top=filedir1+"SA"+i+"/charmm-gui/gromacs/Trial1-100/step9_1.gro")
    #pttraj = pt.load(dcds[count], filedir1+"SA"+i+"/charmm-gui/gromacs/Trial1-100/step9_1.gro")
    lipidSelection = traj.topology.select('resname PALP or resname LAUP or resname STEP or resname MYRP')
    lipidTraj = traj.atom_slice(lipidSelection).center_coordinates()
    lipids.append(lipidTraj)
    C1selection = lipidTraj.topology.select('name C1')
    C1Traj = lipidTraj.atom_slice(C1selection)
    coords = C1Traj.xyz
    finalCoords = coords[len(coords)-1]
    upper_indices = ''
    lower_indices = ''
    count2 = 1
    for x, y, z in finalCoords: #for each C1
        if z > 0:
            if len(upper_indices) == 0:
                upper_indices += ('residue '+str(count2))
            else:
                upper_indices+=(' or residue '+ str(count2))
        else:
            if len(lower_indices) == 0:
                lower_indices+=('residue '+str(count2))
            else:
                lower_indices+=(' or residue '+str(count2))
        count2 +=1
    #print(upper_indices)
    upperLeafSelection = lipidTraj.topology.select(upper_indices)
    upperLeafTraj = lipidTraj.atom_slice(upperLeafSelection)
    lowerLeafSelection = lipidTraj.topology.select(lower_indices)
    lowerLeafTraj = lipidTraj.atom_slice(lowerLeafSelection)
    lipidTrajArray.append([upperLeafTraj, lowerLeafTraj])
    #ptTrajArray.append(pttraj)
    count +=1
print(lipidTrajArray)
print(lipids)

## Nematic Order parameter calculation

In [ ]:
## need to create an array of arrays with the indices of EACH atom in each residue
##[[atom indices for res#1], [atom indices for res#2]] ...etc
nemData = []
count = 0
#surface_areas = ["19"]
for ct, area in enumerate(areas):
    nemOrderIdx = []
    resID = []
    chunk = []
    #t_all = md.load(dcds[count], top=filedir1+"SA"+area+"/charmm-gui/step5_assembly.xplor_ext.psf")
    #t_all = md.load(trajfiles[count], top=filedir+area+"/step9_1.gro")
    upper_pdb = lipidTrajArray[ct][0][-1].save_pdb(filedir_g+area+'/upper_PDB.pdb')
    lower_pdb = lipidTrajArray[ct][1][-1].save_pdb(filedir_g+area+'/lower_PDB.pdb')
    #lip = t_all.topology.select('resname LAUP or resname MYRP or resname PALP or resname STEP')
    #t = t_all.atom_slice(lip)
    pu = md.load_pdb(filedir_g+area+'/upper_PDB.pdb')
    pl = md.load_pdb(filedir_g+area+'/lower_PDB.pdb')
    #p = md.load_pdb(filedir1+"/SA19_last_rm.pdb")
    for leaflet in [pu, pl]
        resNum =pu.n_residues 
        lines = open('PDB.pdb', "r")
        #lines = open(filedir1+"/SA19_last_rm.pdb", "r")
        for line in lines:
            if line[0] == "A":
                #print(line[6:11]) #atom index
                #print(line[23:26])##residue sequence number
                resIDCur = line[23:26]
                #print(resIDCur)
                resID.append(resIDCur)
                atomIdx = int(line[6:11].replace(" ", ""))
                #print(atomIdx)
                if len(resID) > 1:
                    if len(resID) == resNum - 1:
                        chunk.append(atomIdx)
                        nemOrderIdx.append(chunk)
                    #print(resID[-1])
                    #print(resID[-2])
                    #print('END')
                    elif resID[-1] == resID[-2]: #if the last thing added to the resID guy is the same 
                    #as the previous guy added, then there has been no change in the residue, so we should keep 
                    #adding indices to chunk
                        #print('here')
                        chunk.append(atomIdx)
                        #print(chunk)
                    #else, we need to append the chunk to the nematic order index and start a new chunk
                    else:
                        nemOrderIdx.append(chunk)
                        chunk = []
                        chunk.append(atomIdx)
                    #if resSeqCur != resSeq: 
                    #    nemOrderIdx.append(chunk)
                    #else:
                    #    chunk.append(atomIdx)
                else:
                    chunk.append(atomIdx)
        s = md.compute_nematic_order(t, indices = nemOrderIdx)
    nemData.append(s)
    count+=1